In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sksurv.metrics import concordance_index_censored
import itertools

print("--- Starting Deep Learning HYPERPARAMETER TUNING ---")
tf.get_logger().setLevel('ERROR') # Suppress verbose TensorFlow logs

--- Starting Deep Learning HYPERPARAMETER TUNING ---


In [2]:
# ==============================================================================
# --- 1. LOAD YOUR SAVED DATA ---
# ==============================================================================
base_data_dir = "/Users/saifahmed/development/ESCC/data"
processed_dir = os.path.join(base_data_dir, "processed")

X_file = os.path.join(processed_dir, "X_combined_model_input.csv")
y_file = os.path.join(processed_dir, "y_survival_target.csv")

X_combined = pd.read_csv(X_file, index_col=0)
y_survival = pd.read_csv(y_file, index_col=0)

print(f"Loaded X_combined data. Shape: {X_combined.shape}")
print(f"Loaded y_survival data. Shape: {y_survival.shape}")

Loaded X_combined data. Shape: (60, 2838)
Loaded y_survival data. Shape: (60, 2)


In [3]:
# ==============================================================================
# --- 2. PREPARE DATA FOR TENSORFLOW/KERAS ---
# ==============================================================================
X = X_combined.astype(np.float32).values
y_keras = y_survival[['time', 'event']].astype(np.float32).values

# ==============================================================================
# --- 3. DEFINE THE CUSTOM COX SURVIVAL LOSS FUNCTION ---
# ==============================================================================
# (This is the same, correct loss function from before)
def cox_loss(y_true, y_pred):
    indices = tf.argsort(y_true[:, 0], direction='DESCENDING')
    time = tf.gather(y_true[:, 0], indices)
    event = tf.gather(y_true[:, 1], indices)
    risk = tf.gather(y_pred[:, 0], indices)
    
    event_mask = tf.cast(event, dtype=tf.bool)
    risk_observed = tf.boolean_mask(risk, event_mask)
    
    risk_sum = tf.math.cumsum(tf.exp(risk))
    denominator = tf.boolean_mask(risk_sum, event_mask)
    log_denominator = tf.math.log(denominator)
    
    loss = -tf.reduce_sum(risk_observed - log_denominator)
    return loss

In [4]:
# ==============================================================================
# --- 4. SPLIT DATA (Same 80/20 split as before) ---
# ==============================================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y_keras, 
    test_size=0.20, 
    random_state=42, # This ensures our Test set is identical
    stratify=y_keras[:, 1]
)
print(f"\nTraining data: {X_train.shape}, Test data: {X_test.shape}")


Training data: (48, 2838), Test data: (12, 2838)


In [5]:
# ==============================================================================
# --- 5. DEFINE THE "SEARCH SPACE" AND TUNING LOOP ---
# ==============================================================================
print("\nStarting hyperparameter search loop...")

# Define the hyperparameters we want to test
param_grid = {
    'learning_rate': [1e-3, 5e-4, 1e-4],
    'l2_reg': [0.01, 0.1],
    'dropout_rate': [0.3, 0.5]
}

# Create all unique combinations
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

results = []

# This function builds the model based on the parameters
def build_model(input_shape, lr, l2, dropout):
    model = keras.Sequential([
        layers.Input(shape=(input_shape,)),
        layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2)),
        layers.BatchNormalization(),
        layers.Dropout(dropout),
        layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2)),
        layers.BatchNormalization(),
        layers.Dropout(dropout),
        layers.Dense(1, activation='linear') 
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss=cox_loss
    )
    return model

# Loop over all parameter combinations
for i, params in enumerate(all_params):
    print(f"\n--- Testing Combination {i+1}/{len(all_params)} ---")
    print(f"Params: {params}")
    
    dl_model = build_model(
        X_train.shape[1],
        lr=params['learning_rate'],
        l2=params['l2_reg'],
        dropout=params['dropout_rate']
    )
    
    # Train the model
    history = dl_model.fit(
        X_train,
        y_train,
        epochs=150,
        batch_size=16,
        validation_split=0.2, # We use a validation set for early stopping
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)],
        verbose=0 # Turn off the epoch logs to keep output clean
    )
    
    # Evaluate on the TEST set
    dl_risk_scores = dl_model.predict(X_test, verbose=0).flatten()
    test_times = y_test[:, 0]
    test_events = y_test[:, 1].astype(bool)
    
    c_index = concordance_index_censored(
        test_events,
        test_times,
        dl_risk_scores
    )[0]
    
    print(f"Resulting C-Index: {c_index:.4f}")
    params['c_index'] = c_index
    results.append(params)


Starting hyperparameter search loop...

--- Testing Combination 1/12 ---
Params: {'learning_rate': 0.001, 'l2_reg': 0.01, 'dropout_rate': 0.3}
Resulting C-Index: 0.8929

--- Testing Combination 2/12 ---
Params: {'learning_rate': 0.001, 'l2_reg': 0.01, 'dropout_rate': 0.5}
Resulting C-Index: 0.9107

--- Testing Combination 3/12 ---
Params: {'learning_rate': 0.001, 'l2_reg': 0.1, 'dropout_rate': 0.3}
Resulting C-Index: 0.8750

--- Testing Combination 4/12 ---
Params: {'learning_rate': 0.001, 'l2_reg': 0.1, 'dropout_rate': 0.5}
Resulting C-Index: 0.8929

--- Testing Combination 5/12 ---
Params: {'learning_rate': 0.0005, 'l2_reg': 0.01, 'dropout_rate': 0.3}
Resulting C-Index: 0.9286

--- Testing Combination 6/12 ---
Params: {'learning_rate': 0.0005, 'l2_reg': 0.01, 'dropout_rate': 0.5}
Resulting C-Index: 0.9107

--- Testing Combination 7/12 ---
Params: {'learning_rate': 0.0005, 'l2_reg': 0.1, 'dropout_rate': 0.3}
Resulting C-Index: 0.9107

--- Testing Combination 8/12 ---
Params: {'learni

In [6]:
# ==============================================================================
# --- 6. SHOW FINAL RESULTS ---
# ==============================================================================
print("\n--- Hyperparameter Tuning Complete ---")

# Create a DataFrame to see all results
results_df = pd.DataFrame(results).sort_values(by='c_index', ascending=False)

print("All Model Results:")
print(results_df.to_string())

best_c_index = results_df.iloc[0]['c_index']
best_params = results_df.iloc[0].to_dict()

print("\n--- FINAL MODEL COMPARISON ---")
print(f"RSF (Benchmark):          0.8750")
print(f"Deep Learning (Best Tuned): {best_c_index:.4f}")

if best_c_index > 0.8750:
    print("\n--- !!! FINAL SUCCESS !!! ---")
    print("Your *Tuned* Deep Learning model outperformed the Random Survival Forest.")
    print("Best Parameters:")
    print(best_params)
else:
    print("\n--- FINAL RESULT ---")
    print("The Random Survival Forest (0.8750) remains the best-performing model.")


--- Hyperparameter Tuning Complete ---
All Model Results:
    learning_rate  l2_reg  dropout_rate   c_index
9          0.0001    0.01           0.5  0.946429
4          0.0005    0.01           0.3  0.928571
1          0.0010    0.01           0.5  0.910714
5          0.0005    0.01           0.5  0.910714
6          0.0005    0.10           0.3  0.910714
0          0.0010    0.01           0.3  0.892857
3          0.0010    0.10           0.5  0.892857
2          0.0010    0.10           0.3  0.875000
7          0.0005    0.10           0.5  0.875000
8          0.0001    0.01           0.3  0.875000
10         0.0001    0.10           0.3  0.875000
11         0.0001    0.10           0.5  0.821429

--- FINAL MODEL COMPARISON ---
RSF (Benchmark):          0.8750
Deep Learning (Best Tuned): 0.9464

--- !!! FINAL SUCCESS !!! ---
Your *Tuned* Deep Learning model outperformed the Random Survival Forest.
Best Parameters:
{'learning_rate': 0.0001, 'l2_reg': 0.01, 'dropout_rate': 0.5, 'c_ind